In [2]:
import pyomo as pyo
import pandas as pd
import numpy as np
from data_processing import load_all_data
from calcula_fpo import calcular_fpo

In [3]:
data = load_all_data()
D_GEN = data['D_GEN']
D_LIN = data['D_LIN']
D_LOAD = data['D_LOAD']

In [4]:
#Deixando os dados em horas e nos formatos conhecidos
#Dados de Geração
D_GEN['Lambda'] = D_GEN['Falhas/ano'] / 8760 # 8760 horas em um ano
D_GEN['MTTF'] = 1/D_GEN['Lambda'] #Tempo médio entre falhas
D_GEN['mu'] = 1/D_GEN['Tempo de reparo(h)'] #Taxa de reparo/hora
D_GEN['A'] = D_GEN['mu']/(D_GEN['mu'] + D_GEN['Lambda']) #Disponibilidade
D_GEN['U'] = 1 - D_GEN['A'] #Indisponibilidade

#Dados de Linha
D_LIN['Lambda'] = D_LIN['Falhas/(ano.milha)'] * D_LIN['Comprimento'] / 8760 # 8760 horas em um ano
D_LIN['MTTF'] = 1 / D_LIN['Lambda'] #Tempo médio entre falhas
D_LIN['mu'] = 1 / D_LIN['Tempo de Reparo'] #Taxa de reparo/hora
D_LIN['A'] = D_LIN['mu'] / (D_LIN['mu'] + D_LIN['Lambda']) #Disponibilidade
D_LIN['U'] = 1 - D_LIN['A'] #Indisponibilidade

#Numero de componentes
Ng = len(D_GEN)
Nl = len(D_LIN)
Nc = len(D_LOAD)


In [5]:
def calcular_estatisticas(resultados):
    if len(resultados) > 1:
        # Calcular a média
        media = sum(resultados) / len(resultados)
        
        # Calcular a variância (usando a fórmula da variância amostral)
        variancia = sum((x - media) ** 2 for x in resultados) / (len(resultados) - 1)  # ddof=1 para variância amostral
        
        # Calcular a variância dividida pelo tamanho da lista
        variancia_media = variancia / len(resultados)
        
        # Calcular o Beta
        beta = (variancia_media ** 0.5) / media if media != 0 else 10
        return media, variancia, beta
    else:
        return 10, 10, 10

In [ ]:
# Variáveis de controle para o loop de Monte Carlo
NS = 0                     # Contador de simulações
NSmax = 20000              # Limite de simulações

beta = 10                  # Convergência de LOLP
beta_tol = 0.01            # Tolerância de LOLP

beta_EPNS = 10             # Convergência de EPNS
beta_EPNS_tol = 0.02       # Tolerância de EPNS

# Variáveis de acumulação para LOLP e EPNS
SLOLP, SLOLP2 = 0, 0       # Somas de LOLP e quadrado
SEPNS, SEPNS2 = 0, 0       # Somas de EPNS e quadrado

tempo_falha_total = []

# Loop de Monte Carlo
while (beta >= beta_tol or beta_EPNS >= beta_EPNS_tol) and NS <= NSmax:
    NS += 1
    # Geração de sorteios aleatórios para geradores e linhas
    sorteio_ger = np.random.rand(Ng)
    sorteio_linhas =  np.random.rand(Nl)
    estado_ger = [1 if sorteio_ger[i] > D_GEN['U'][i] else 0 for i in range(Ng)]
    estado_linhas = [1 if sorteio_linhas[i] > D_LIN['U'][i] else 0 for i in range(Nl)]

    # Verificação de perda de carga: Se corte > 0, há perda de carga
    corte = calcular_fpo(estado_ger, estado_linhas, False)

    if corte > 0:
        #Atualização das somas do LOLP e EPNS
        SLOLP += 1
        SLOLP2 += 1**2
        SEPNS += corte
        SEPNS2 += corte**2

        ger_falhos = [i for i in range(Ng) if estado_ger[i] == 0]
        lin_falhos = [i for i in range(Nl) if estado_linhas[i] == 0]

        # Sorteando o tempo de reparo para cada componente falho (exponencial com taxa mu)
        tempos_reparo_geradores = {i: np.random.exponential(1/D_GEN['mu'][i]) for i in ger_falhos}
        tempos_reparo_linhas = {i: np.random.exponential(1/D_LIN['mu'][i]) for i in lin_falhos}
        tempo_falha = 0
        
        while corte > 0:
            menor_tempo_ger = np.inf
            menor_tempo_lin = np.inf

            # Definindo o menor tempo de reparo e o componente correspondente
            if tempos_reparo_geradores: # Verifica se o dicionário não está vazio
                menor_tempo_ger = min(tempos_reparo_geradores.values())
                componente_menor_tempo_ger = min(tempos_reparo_geradores, key=tempos_reparo_geradores.get)
            if tempos_reparo_linhas: # Verifica se o dicionário não está vazio
                menor_tempo_lin = min(tempos_reparo_linhas.values())
                componente_menor_tempo_lin = min(tempos_reparo_linhas, key=tempos_reparo_linhas.get)

            #Escolhendo o menor tempo de reparo:
            if menor_tempo_ger < menor_tempo_lin:
                tempo_falha += menor_tempo_ger
                # Atualizar o estado do gerador e remover o tempo de reparo do dicionário
                estado_ger[componente_menor_tempo_ger] = 1
                del tempos_reparo_geradores[componente_menor_tempo_ger]
            else:
                tempo_falha += menor_tempo_lin
                # Atualizar o estado da linha e remover o tempo de reparo do dicionário
                estado_linhas[componente_menor_tempo_lin] = 1
                del tempos_reparo_linhas[componente_menor_tempo_lin]
            #Calcular o corte
            corte = calcular_fpo(estado_ger, estado_linhas, False)
        tempo_falha_total.append(tempo_falha)
    
    # Cálculo de convergência após 200 iterações
    if NS > 100:
        # Cálculo de LOLP e beta
        ELOLP = SLOLP / NS
        varLOLP = (SLOLP2 - NS * ELOLP**2) / (NS - 1)
        varELOLP = varLOLP / NS
        beta = np.sqrt(varELOLP) / ELOLP if ELOLP != 0 else 0

        # Cálculo de EPNS e beta_EPNS
        EPNS = SEPNS / NS
        varEPNS = (SEPNS2 - NS * EPNS**2) / (NS - 1)
        varEPNS = varEPNS / NS
        beta_EPNS = np.sqrt(varEPNS) / EPNS if EPNS != 0 else 0

#Obtenção do EPNS:
EPNS_MC = EPNS
#Obtenção do LOLP:
LOLP_MC = ELOLP
#Cálculo do EENS:
EENS_MC = EPNS * 8760
#Cálculo do LOLE:
LOLE_MC = 8760 * ELOLP
#Cálculo do LOLD:
LOLD_MC = sum(tempo_falha_total)/len(tempo_falha_total)
#Cálculo do LOLF:
LOLF_MC = LOLE_MC / LOLD_MC

In [26]:
# Print dos Resultados:
print(f"Expected Energy Not Supplied (EENS): {EENS_MC/1000:.2f} GWh/ano")
print(f"Expected Power Not Supplied (EPNS): {EPNS_MC:.2f} MW")
print(f"Loss of Load Duration (LOLD): {LOLD_MC:.2f} horas/eventos de corte")
print(f"Loss of Load Expectation (LOLE): {LOLE_MC:.2f} horas/ano")
print(f"Loss of Load Probability (LOLP): {LOLE_MC*100:.2f}  %")
print(f"Loss of Load Frequency (LOLF): {LOLF_MC:.2f} eventos de corte/ano")

Expected Energy Not Supplied (EENS): 51.56 GWh/ano
Expected Power Not Supplied (EPNS): 5.89 MW
Loss of Load Duration (LOLD): 36.95 horas/eventos de corte
Loss of Load Expectation (LOLE): 775.18 horas/ano
Loss of Load Probability (LOLP): 77518.25  %
Loss of Load Frequency (LOLF): 20.98 eventos de corte/ano
